### 两种协议
python中的协议是一种非正式接口,是隐含的按照约定定义,比如一个序列协议,python对象的表现就的像一个序列需要提供的方法(鸭子类型).

- 动态协议:由解析器支持(鸭子类型)
- 静态协议:3.8的typing.Protocl约定

In [5]:
class Vowels:

    def __getitem__(self,i):
        return 'ABCD'[i]

v= Vowels()
print(v[1])
'B' in v # 会通过调用__getitem__特殊方法来支持

B


True

### 鸭子类型利用
1. python喜欢序列:
    
    python的数据模型的哲学是尽量支持基本的动态协议,对序列来说即便是最简单的也会力求做到最好.如上面的Vowels对象,没有继承abc.Sequence只实现了__getitem__方法都支持部分序列功能,没有__contains__方法也能使用in运算符.

    >python解释器和list,str等内置序列根本不依赖那个抽象,collection.abc.MutableSequence为了说明序列该支持什么操作(mutable 易变)

    >collections.abc模块中的大多数抽象基类存在的目的是确立由内置对象实现并且由解释器隐式支持的接口

2. 猴子补丁实现运行协议

    在运行时修改类或者模块的代码,而不动源代码
    
3. 防御性编程和快速失败
    对于动态语言而言,快速失败可以提高程序的安全性,让程序更易于维护,快速失败尽可能早抛出错误

    技巧:

     - 如果函数是一个可迭代对象,按照列表处理就不通过类型检查强制要求传入一个列表list(args)
     - 如果可迭代对象数据太多或者像shuffle这样原地处理函数,list()这种方式就不使用,应该用isinstance(obj,abc.MutableSequence)
     - 如果传入是无穷生成器,则可先用len()先获取长度(遇到无效参数直接报错)
    


In [15]:
from collections.abc import *
from random import shuffle
# shuffle 洗牌
# 猴子补丁
class Card:

    def __init__(self) :
        self._arr=[i for i in range(10)]
    
    def __iter__(self):
        return self._arr
    
    def __getitem__(self,i):
        return self._arr[i]
    
    def __len__(self):
        return len(self._arr)

def set_item(self,i,v):
    self._arr[i]=v

card=Card()
Card.__setitem__=set_item

shuffle(card)        

### 大鹅类型
python中没有interface接口,通过使用抽象基类定义接口,在运行时显式检查.抽象基类时对鸭子类型的补充,提供了一种定义接口的方式.相比使用hasattr()方式识别,抽象基类可以使用isinstance()和issubclass()来识别.**抽象基类引入了虚拟子类,这种类不继承其他类却能被isinstance()和issubclass()识别**

> 抽象基类最重要的优势,使用register类方法在代码中把某个类声明为一个抽象基类的虚拟子类

In [11]:
from collections.abc import (MutableSequence,Sized)

hasattr(MutableSequence,'__getitem__')

# 抽象基类的本质就是实现特殊方法
class Len:
    def __len__(self):return 1

# Sized __subclasshook__方法 hook钩子
print(issubclass(Len,Sized))
l=Len()
print(isinstance(l,Sized))

# 如果子类化一个抽象基类没有实现它的方法,那在运行时将会抛出TypeError
class Deck(MutableSequence):
    def __init__(self):
        self.arr=[None]*10
    
    def insert(self, index, value ):
         self.arr[index] =value

    def __delitem__(self,index):
        del self.arr[index]

    def __setitem__(self,index,val):
        self.arr[index]=val
    
    def __len__(self):
        return len(self.arr)

    def __getitem__(self,index):
        return self.arr[index]
    
    def __repr__(self):
        return ','.join((f'{val}' for val in self.arr))
d=Deck()
d.insert(0,1)
print(d)

True
True
1,None,None,None,None,None,None,None,None,None


### 标准库中的抽象基类
标准库的抽象基类都在collection.abc中,io和numbers包中也有一些抽象基类.另外每个抽象基类都依赖于abc模块中ABC类

- Iterable,Container,Sized:分别通过`__iter__`,`__contains__`,`__len__`支持
- Collection:自身没方法
- Sequence,Mapping,Set:主要是不可变容器,都有对应的Mutable***容器
- MappingView:映射方法items(),keys(),values()分别返回实现了ItemsView,KeysView,ValueView
- Iterator:Iterable子类
- Callable,Hashable:可调用,可哈希

> 使用isinstance检查Iterable,Hashable可能不准确,isinstanc(obj,Hashable)obj可能是不可哈希元组,最好使用hash(),isinstance(obj,Iterable)可能返回false,obj对象可以通过__getitem__特殊方法来迭代,最好调用iter()

### 抽象基类语法详解
声明抽象基类的标准方式时继承abc.ABC或其他抽象基类,除了ABC基类和@abstracmethod装饰器,abc模块还定义了@abstractclassmethod,@abstractstaticmethod,@abstractproperty装饰器,后面3个装饰器在3.3中已经弃用.**@absractmethod叠放装饰器的顺序很重要,它应该放在最里面**

In [5]:
from abc import ABC,abstractmethod

class AbstractWork(ABC):
    def __init__(self):
        self._arr=[None]*10

    @abstractmethod
    def add_work(self,i):
        print('supper')
        ...

    def display(self):
        for i in self._arr:
            print(i)    
    
class Work(AbstractWork):

    def add_work(self, i):
        return self._arr.append(i)

w=Work()
w.add_work('s')

### 抽象基类的虚拟子类
**大鹅类型的基本特征是即便不继承,也有办法把一个类注册为抽象基类的虚拟子类**.这样做会承诺注册的类忠实地实现了抽象类定义的接口,python会相信我们,不在检查,但是如果是错误的注册在运行时会抛出异常.注册虚拟子类的方式时在抽象基类上调用register类方法,这么做之后,注册的类就变成了抽象基类的虚拟子类,而且issubclass函数也能识别这个关系,但是注册的类不会从它抽象基类中基础任何方法或属性.(另外register可以当做装饰器调用)

### __subclasshook__方法注册虚拟子类

In [16]:
from abc import ABC
class A(ABC):

    @classmethod
    def __subclasshook__(cls,C):
        if any("__a__" in B.__dict__ for B in C.__mro__):
            return True
        return NotImplemented # 返回NotImplemented,让子类继续查下去
    
# 必须是大鹅类才能将它注册为虚拟子类
@A.register
class B:
    pass

class MyA:
    def __a__(self):
        ...

issubclass(MyA,A)
     

True

### 静态协议
使用继承typing.Protocol实现静态鸭子类型,可以通过@runtime_checkable装饰器让协议支持在运行时使用isinstance/issubclass检查(背后原因typing.Protocol支持__subclasshook__)

> 充分利用鸭子类型
> ```
> isinstance(o,(complex,SupportsComlex))
> isinstance(o,numbers.Complex)
> try:
>    c=complex(v)
> except TypeError as exc
>    raise TypeError('')
> ```

### 运行时协议检查的局限性
比如实现__float__方法的类在运行时被认定为SupportFloat的虚拟子类,不管__float__方法返回是一个float值.类似的isinstance或issubclass只检查有没有特点的方法,不检查方法的签名,更不会检查方法的类型注解

### 实现一个静态协议
创建一个类然后继承typing.Protocol类,添加协议方法

In [18]:
from typing import Protocol,runtime_checkable

@runtime_checkable # 支持运行时检查,可以使用isinstance,issubclass进行检查
class SupportLen(Protocol):
    def __len__(self):...

arr=[]

isinstance(arr,SupportLen)

True